# Deep Q-Learning Workshop

## Einführung

2013 zeigte DeepMind wie sie mit ein und denselben Machine Learning Algorithmus lernen können viele verschiedene Atari Spiele zu spielen. Diese Algorithmus ist das *Deep-Q-Learning*. Wie gut er Spiele lernen kann zeigt DeepMind gerne anhand von Breakout.

In [ ]:
from IPython.display import HTML
HTML('<iframe width="800" height="560" src="https://www.youtube.com/embed/TmPfTpjtdgg?rel=0&amp;showinfo=0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

In diesem Workshop werden wir diesen Algorithmus kennen lernen, zunächst für einige einfache Spiele umsetzen und uns zum Schluss Breakout vornehmen. 

Als erstes sehen wir uns das OpenAI Gym genauer an. Dies ist die Simulation in welcher neben den Atari Spielen auch physikalische Experimente durchgeführt werden können. Dann werden wir *Deep-Q-Learning* implementieren und lernen einen Stab zu balancieren oder ein doppeltes Pendel nach oben zu schwingen. Daraufhin werden wir noch zwei Verbesserungen implementieren, den *Experience Replay Memory* und *Double-DQN*. Damit lassen wir den Algorithmus lernen einen SpaceX Booster auf dem Droneship zu landen. Zuletzt setzen wir das *Deep* in *Deep-Q-Learning* um und erstellen ein Modell, das nur von den Pixeln des Spieles Atari Breakout lernt, dieses zu spielen.

### Die Paper

Den Deep-Q-Learning Algorithmus hat DeepMind in dem Paper [Playing Atari with Deep Reinforcement Learning](http://arxiv.org/abs/1312.5602) vorgestellt. Später wurde dieser mit der Double-DQN Verbesserung in einem Paper in dem Nature Magazin veröffentlicht: [Human-level control through deep reinforcement learning](http://dx.doi.org/10.1038/nature14236).

#### Weiterführend
Über die Jahre wurden viele kleine Weiterentwicklungen des DQN-Algorithmus vorgestellt. Die vielversprechenden sind in dem [Rainbow: Combining Improvements in Deep Reinforcement Learning](http://arxiv.org/abs/1710.02298) Paper zusammengefasst und dieses Paper ist sicher ein guter Einstiegspunkt, um sich in diese Weiterentwicklungen zu vertiefen.


## Imports

Wir benutzen einige Packages. Bitte stell sicher, dass alle Packages wie in der Readme beschrieben installiert wurden.

In [ ]:
# The gym
import gym
from gym.wrappers import Monitor

# The typical imports
import numpy as np
import itertools
import glob
import re

# Deep Learning Tools
import keras
from keras import backend as K
from keras.layers.merge import multiply

# Displaying Tools
import matplotlib.pyplot as plt
%matplotlib notebook
from tqdm import tqdm_notebook as tqdm
from ipywidgets import widgets
from IPython.display import display
from matplotlib import animation
from JSAnimation.IPython_display import display_animation


# Utils from other notebooks
import import_ipynb
import Ringbuffer
import utils

## Das OpenAi Gym kennen lernen

[OpenAI Gym](https://gym.openai.com/)

In [ ]:
environment = 'CartPole-v1'
env = gym.make(environment)

In [ ]:
env.metadata

In [ ]:
env.action_space

In [ ]:
env.observation_space

In [ ]:
# Create a breakout environment
# Reset it, returns the starting frame
frames = []
frame = env.reset()

In [ ]:
frame

In [ ]:
is_done = False
while not is_done:
  # Perform a random action, returns the new frame, reward and whether the game is over
  frame, reward, is_done, _ = env.step(env.action_space.sample())
  # Render
  frames.append(env.render(mode = 'rgb_array'))
env.close()

In [ ]:
utils.display_frames_as_gif(frames)

### Selbst den Stab balancieren

Mit dem folgenden Code kannst du selbst einmal probieren den Stab zu balancieren. Unter der Grafik werden zwei Buttons für die zwei möglichen Aktionen angezeigt. Jedes mal wenn du einen Button klickst, wird ein Schritt in der Simulation mit der entsprechenden Aktion durchgeführt. Sobald der Stab zu weit kippt wird die Simulation automatisch neu gestartet.

In [ ]:

def leftclicked(something):
    """ Apply a force to the left of the cart"""
    onclick(0)

def rightclicked(something):
    """ Apply a force to the right of the cart"""
    onclick(1)
    
def display_buttons():
    """ Display the buttons you can use to apply a force to the cart """
    left = widgets.Button(description="<")
    right = widgets.Button(description=">")
    display(left, right)
    
    left.on_click(leftclicked)
    right.on_click(rightclicked)

# Create the environment and display the initial state
env = gym.make('CartPole-v0')
observation = env.reset()
firstframe = env.render(mode = 'rgb_array')
fig,ax = plt.subplots()
im = ax.imshow(firstframe) 

# Show the buttons to control the cart
display_buttons()


# Function that defines what happens when you click one of the buttons
frames = []
def onclick(action):
    global frames
    observation, reward, done, info = env.step(action)
    frame = env.render(mode = 'rgb_array')
    im.set_data(frame)
    frames.append(frame)
    print("Observation: {}. Reward: {}. Done: {}. Info: {}.".format(observation, reward, done, info), end='\r')
    if done:
        env.reset()
        print("\nDone!")

Nun können wir uns deinen Balanceversuch noch etwas flüssiger anschauen:

In [ ]:
utils.display_frames_as_gif(frames)

## Q-Learning Intuition

TODO
Hier sollte ein grundlegendes Verständnis für den DQN Algorithmus und einige Begriffe verschafft werden.


Diese vier evtl schon oben beim Gym.

- agent
- environment
- action
- state


- policy
- discounting
- optimal action value function Q*(s,a)
- Bellman equation
- model-free
- e-greedy strategy
- experience replay



## Die Einzelteile

Wie wir in der Q-Learning Intution gesehen haben besteht der Algorithmus aus mehreren einzelteilen. Diese wollen wir nun implementieren.

### Die Q-Value Approximation Function
Oder auch das Machine Learning Modell

In [ ]:
def gym_model(n_actions, input_shape):
    
    # With the functional API we need to define the inputs.
    frames_input = keras.layers.Input(input_shape, name='frames')
    actions_input = keras.layers.Input((n_actions,), name='mask')

    flat = keras.layers.Flatten()(frames_input)
    hidden = keras.layers.Dense(16, activation='relu')(flat)
    hidden = keras.layers.Dense(16, activation='relu')(hidden)
    hidden = keras.layers.Dense(16, activation='relu')(hidden)
    output = keras.layers.Dense(n_actions)(hidden)

    # Finally, we multiply the output by the mask!
    filtered_output = multiply([output, actions_input])

    model = keras.models.Model(inputs=[frames_input, actions_input], outputs=filtered_output)
    optimizer = optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.95, epsilon=0.01)
    model.compile(optimizer, loss="mse")
    return model

In the paper [Human-level control through deep reinforcement
learning](http://www.davidqiu.com:8888/research/nature14236.pdf) DeepMind suggests using a separate network to predict target values and only update that network every 10000 iterations. 

In [ ]:
def copy_model(model):
    model_copy = keras.models.clone_model(model)
    model_copy.set_weights(model.get_weights())
    return model_copy

### Die Epsilon Greedy Strategie

In [ ]:
def get_epsilon_for_iteration(iteration, max_iteration, max_q, min_q):
    if(iteration >= max_iteration):
        return min_q
    else:
        return ((min_q - max_q) / max_iteration) * iteration + max_q

In [ ]:
get_epsilon_for_iteration(0, 1000000, 1.0, 0.1)

In [ ]:
get_epsilon_for_iteration(999999,  1000000, 1.0, 0.1)

In [ ]:
get_epsilon_for_iteration(1000000,  1000000, 1.0, 0.1)

### Die Wahl der richtigen Aktion

In [ ]:
def choose_best_action(model, state, action_space_size):
    # Make a "batch" out of the state.
    state_ext = np.expand_dims(state, axis=0)
    # Create a mask consisting out of ones for the model imput.
    action_mask = np.ones((state.shape[-1], action_space_size))
    logits = model.predict([state_ext,action_mask])[0]
    # TODO: do not use argmax but multinomal (needs a softmax layer?)
    return np.argmax(logits)

In [ ]:
def e_greedy_choose_best_action(model, state, action_space):
    if np.random.random() < epsilon:
        action = action_space.sample()
    else:
        action = choose_best_action(model, state, action_space.n)
    return action
        

In [ ]:
def one_hot_encode(action, action_space_size):
    return keras.utils.np_utils.to_categorical(action, num_classes=action_space_size)

### Der State

The paper suggests to input the last four frames to the network, so it can figure out what the speed, directetion and acceleration of an object is. At the beginning of each game, we will initialise a buffer with 4 times the very first frame. Each game step we will remove the oldest frame and append the new frame. To do that we will implement functions that help updating the buffer.

In [ ]:
def init_state_buffer(size, frame):
    return np.stack([frame] * size, axis = 1)

def update_state_buffer(state_buffer, new_frame):
    return np.append(state_buffer[:,1:], np.expand_dims(new_frame, axis=1), axis = 1)

In [ ]:
def preprocess(state):
    return state

# Training Loop

### Hyperparams

In [ ]:
MINIBATCH_SIZE = 32
REPLAY_MEMORY_SIZE = 50000 
AGENT_HISTORY_LENGTH = 4
DISCOUNT_FACTOR = 0.99
ACTION_REPEAT = 1 
REPLAY_START_SIZE = 100
TARGET_MODEL_UPDATE_INTERVALL = 100
MAXIMUM_ITERATIONS = 50000

### Init

In [ ]:
env = gym.make(environment)
memory = RingBuf(REPLAY_MEMORY_SIZE)
model = gym_model(env.action_space.n, (env.observation_space.shape[0],AGENT_HISTORY_LENGTH))
target_model = copy_model(model)
restart = False
iteration = 0
start_episode = 0
reward_history = []

### Fill Replay Memory with random actions

Or use a models decisions if we restart training from a saved model.

In [ ]:
first_state = preprocess(env.reset())
state = init_state_buffer(AGENT_HISTORY_LENGTH, first_state)

for _ in tqdm(range(REPLAY_START_SIZE)):
    if not restart:
        action = env.action_space.sample()
    else: 
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = choose_best_action(model, state, env.action_space.n)
        
    one_hot_action = keras.utils.np_utils.to_categorical(action, num_classes=env.action_space.n)

    for _ in range(ACTION_REPEAT):
        new_frame, reward, is_done, _ = env.step(action)
        new_frame = preprocess(new_frame)
        next_state = update_state_buffer(state, new_frame)
        if is_done:
            break
    memory.add(state, one_hot_action, next_state, reward, is_done)

    state = next_state
    if is_done:
        first_state = preprocess(env.reset())
        state = init_state_buffer(AGENT_HISTORY_LENGTH, first_state)


### Train Model while playing

This function is from the blogpost.

In [ ]:
def fit_batch(model, traget_model, gamma, start_states, actions, rewards, next_states, is_terminal):
    """Do one deep Q learning iteration.
    
    Params:
    - model: The DQN
    - traget_model: The target DQN
    - gamma: Discount factor (should be 0.99)
    - start_states: numpy array of starting states of shape [batch_size, 105, 80, 4]
    - actions: numpy array of one-hot encoded actions corresponding to the start states 
                of shape [batch_size, number_of_possible_actions]
    - rewards: numpy array of rewards corresponding to the start states and actions
    - next_states: numpy array of the resulting states corresponding to the start states 
                    and actions of shape [batch_size, 105, 80, 4]
    - is_terminal: numpy boolean array of whether the resulting state is terminal
    
    """
    # First, predict the Q values of the next states. Note how we are passing ones as the mask.
    next_Q_values = traget_model.predict([next_states, np.ones(actions.shape)])

    # The Q values of each start state is the reward + gamma * the max next state Q value
    #     or just reward when it is a terminal state
    Q_values = rewards + np.invert(is_terminal).astype(np.float32) * gamma * np.max(next_Q_values, axis=1)
    # Fit the keras model. Note how we are passing the actions as the mask and multiplying
    # the targets by the actions.
    model.fit(
        [start_states, actions], actions * Q_values[:, None],
        epochs=1, batch_size=len(start_states), verbose=0
    )

In [ ]:
def q_iteration(env, model, target_model, gamma, batch_size, iteration, state, replay_memory):
    # Choose epsilon based on the iteration
    epsilon = get_epsilon_for_iteration(iteration, 1000, 1.0, 0.1)
    
    if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = choose_best_action(model, state, env.action_space.n)
    one_hot_action = keras.utils.np_utils.to_categorical(action, num_classes=env.action_space.n)

    for _ in range(ACTION_REPEAT):
        new_frame, reward, is_done, _ = env.step(action)
        new_frame = preprocess(new_frame)
        next_state = update_state_buffer(state, new_frame)
        if is_done:
            break
        
    memory.add(state, one_hot_action, next_state, reward, is_done)
    state = next_state
    
    # Sample and fit
    states_batch, action_batch, next_states_batch, reward_batch, done_batch = memory.sample_batch(batch_size)
    fit_batch(model,target_model, gamma, states_batch, action_batch, reward_batch, next_states_batch, done_batch)
    
    return state, is_done, reward

In [ ]:
env = gym.make(environment)
env = Monitor(env,
                  directory=monitor_path,
                  resume=True,
                  video_callable=lambda count: count % record_video_every ==0)

fig,ax = plt.subplots(1,1)

for episode in tqdm(itertools.count(start=start_episode)):
    total_reward = 0
    is_done = False
    first_state = preprocess(env.reset())
    state = init_state_buffer(AGENT_HISTORY_LENGTH, first_state)

    while not is_done:
        state, is_done, reward = q_iteration(env, model, target_model, DISCOUNT_FACTOR, MINIBATCH_SIZE, iteration, state, memory)
        total_reward += reward
        iteration += 1
        
        if iteration % TARGET_MODEL_UPDATE_INTERVALL == 0:
            target_model = copy_model(model)
        
    print("Finished game: {} with reward: {}. Replay Memory Size: {}. Iteration: {}".format(episode + 1, total_reward, len(memory), iteration), end='\r')
    reward_history.append(total_reward)
    
    ax.clear()
    ax.plot(reward_history[-1000:])
    fig.canvas.draw()
    
    if episode % save_model_every == 0:
        model.save_weights("./models/ep_{}_itr_{}.hdf5".format(episode, iteration))
        np.savetxt("./reward_history.csv", reward_history, fmt="%d", delimiter=",")
    if iteration >= MAXIMUM_ITERATIONS:
        break

In [ ]:
!mkdir models
model.save("./models/final.hdf5")

In [ ]:
plt.plot(reward_history)

# Play a game

In [ ]:

# Create a breakout environment
env = gym.make(environment)
# Reset it, returns the starting frame
frames = []
frame = env.reset()

frame_buffer = init_state_buffer(4, preprocess(frame))

is_done = False
while not is_done:
    action = choose_best_action(0.0, model, frame_buffer, env.action_space.n)

    frame, reward, is_done, _ = env.step(action)
    # Render
    frame_buffer = update_state_buffer(frame_buffer, preprocess(frame))
    frames.append(env.render(mode = 'rgb_array'))
env.close()
display_frames_as_gif(frames)
